In [40]:
import numpy as np
import pandas as pd

In [56]:
size = 2
global_n = 4
local_n = int(global_n / size)
ndof = int((local_n + 2) * (global_n + 2))
s_ndof = int((global_n + 2)**2)
energy = 2
n_it = 2
L = 1
h = L / (global_n + 1)
h2 = h * h

In [57]:
def ind(i, j):
    return (i) * (global_n + 2) + (j)

# Serial

In [58]:
unew = np.zeros((s_ndof))
uold = np.zeros((s_ndof))
f = np.zeros((s_ndof))

# Determina a posição dos pontos fonte
s1x = int(1 + global_n / 4)
s1y = s1x

s2x = int(1 + 3 * global_n / 4)
s2y = s2x

print("s1x:{},s1y:{},s2x:{},s2y:{}".format(s1x, s1y, s2x, s2y))

# Injeta calor nos pontos fonte
f[ind(s1x, s1y)] = energy
f[ind(s2x, s2y)] = -energy

error = 0.0

for it in range(n_it):
    # Cada processo deve trocar as bordas com seus vizinhos
    local_error = 0.0

    # Calculo do Stencil
    for i in range(1, global_n + 1):
        for j in range(1, global_n + 1):
            # 2nd order finite difference
            unew[ind(i, j)] = 0.25 * (
                uold[ind(i - 1, j)]
                + uold[ind(i + 1, j)]
                + uold[ind(i, j - 1)]
                + uold[ind(i, j + 1)]
                + h2 * f[ind(i, j)]
            )
            # calculo do erro local
            local_error += (unew[ind(i, j)] - uold[ind(i, j)]) * (
                unew[ind(i, j)] - uold[ind(i, j)]
            )

    error = np.sqrt(local_error)

    print("iter: {}   erro: {}\n".format(it, error))

    tmp = unew
    unew = uold
    uold = tmp  # swap arrays

    print("unew: \n", pd.DataFrame(unew.reshape((global_n + 2, global_n + 2))))
    print("f: \n", pd.DataFrame(f.reshape((global_n + 2, global_n + 2))))
    print("=" * 12)

s1x:2,s1y:2,s2x:4,s2y:4
iter: 0   erro: 0.02828427124746191

unew: 
      0    1    2    3    4    5
0  0.0  0.0  0.0  0.0  0.0  0.0
1  0.0  0.0  0.0  0.0  0.0  0.0
2  0.0  0.0  0.0  0.0  0.0  0.0
3  0.0  0.0  0.0  0.0  0.0  0.0
4  0.0  0.0  0.0  0.0  0.0  0.0
5  0.0  0.0  0.0  0.0  0.0  0.0
f: 
      0    1    2    3    4    5
0  0.0  0.0  0.0  0.0  0.0  0.0
1  0.0  0.0  0.0  0.0  0.0  0.0
2  0.0  0.0  2.0  0.0  0.0  0.0
3  0.0  0.0  0.0  0.0  0.0  0.0
4  0.0  0.0  0.0  0.0 -2.0  0.0
5  0.0  0.0  0.0  0.0  0.0  0.0
iter: 1   erro: 0.012247448713915893

unew: 
      0    1     2    3     4    5
0  0.0  0.0  0.00  0.0  0.00  0.0
1  0.0  0.0  0.00  0.0  0.00  0.0
2  0.0  0.0  0.02  0.0  0.00  0.0
3  0.0  0.0  0.00  0.0  0.00  0.0
4  0.0  0.0  0.00  0.0 -0.02  0.0
5  0.0  0.0  0.00  0.0  0.00  0.0
f: 
      0    1    2    3    4    5
0  0.0  0.0  0.0  0.0  0.0  0.0
1  0.0  0.0  0.0  0.0  0.0  0.0
2  0.0  0.0  2.0  0.0  0.0  0.0
3  0.0  0.0  0.0  0.0  0.0  0.0
4  0.0  0.0  0.0  0.0 -2.0  0

# Paralelo

In [69]:
matriz_dict = {
    0: {"unew": np.zeros((ndof)), "uold": np.zeros((ndof)), "f": np.zeros((ndof))},
    1: {"unew": np.zeros((ndof)), "uold": np.zeros((ndof)), "f": np.zeros((ndof))},
}


error = 0.0

for it in range(n_it):
    for rank in range(size):
        print("Process {}\n".format(rank))

        # Determina a posição dos pontos fonte
        s1x = int(global_n / 4)
        s1y = s1x

        s2x = int(3 * global_n / 4)
        s2y = s2x

        # Determine os pontos de início e fim de cada processo
        y_i = rank * local_n
        y_f = (rank + 1) * local_n

        print("s1x:{},s1y:{},s2x:{},s2y:{}".format(s1x, s1y, s2x, s2y))
        print("y_i:{},y_f:{}".format(y_i, y_f))

        # Injeta calor nos pontos fonte
        if s1y >= y_i and s1y < y_f:
            matriz_dict[rank]["f"][
                ind(s1x + 1 - rank * local_n, s1y + 1)
            ] = energy
        if s2y >= y_i and s2y < y_f:
            matriz_dict[rank]["f"][
                ind(s2x + 1 - rank * local_n, s2y + 1)
            ] = -energy

        # Cada processo deve trocar as bordas com seus vizinhos
        local_error = 0.0

        # Calculo do Stencil
        for i in range(1, local_n + 1):
            for j in range(1, global_n + 1):
                # 2nd order finite difference
                matriz_dict[rank]["unew"][ind(i, j)] = 0.25 * (
                    matriz_dict[rank]["uold"][ind(i - 1, j)]
                    + matriz_dict[rank]["uold"][ind(i + 1, j)]
                    + matriz_dict[rank]["uold"][ind(i, j - 1)]
                    + matriz_dict[rank]["uold"][ind(i, j + 1)]
                    + h2 * matriz_dict[rank]["f"][ind(i, j)]
                )
                # calculo do erro local
                local_error += (
                    matriz_dict[rank]["unew"][ind(i, j)]
                    - matriz_dict[rank]["uold"][ind(i, j)]
                ) * (
                    matriz_dict[rank]["unew"][ind(i, j)]
                    - matriz_dict[rank]["uold"][ind(i, j)]
                )

        error = np.sqrt(local_error)

        print("iter: {}   erro: {}\n".format(it, error))

        tmp = matriz_dict[rank]["unew"]
        matriz_dict[rank]["unew"] = matriz_dict[rank]["uold"]
        matriz_dict[rank]["uold"] = tmp  # swap arrays

        print(
            "unew: \n",
            pd.DataFrame(
                matriz_dict[rank]["unew"].reshape((local_n + 2, global_n + 2))
            ),
        )
        print(
            "f: \n",
            pd.DataFrame(matriz_dict[rank]["f"].reshape((local_n + 2, global_n + 2))),
        )
        print("=" * 12)

Process 0

s1x:1,s1y:1,s2x:3,s2y:3
y_i:0,y_f:2
iter: 0   erro: 0.020000000000000004

unew: 
      0    1    2    3    4    5
0  0.0  0.0  0.0  0.0  0.0  0.0
1  0.0  0.0  0.0  0.0  0.0  0.0
2  0.0  0.0  0.0  0.0  0.0  0.0
3  0.0  0.0  0.0  0.0  0.0  0.0
f: 
      0    1    2    3    4    5
0  0.0  0.0  0.0  0.0  0.0  0.0
1  0.0  0.0  0.0  0.0  0.0  0.0
2  0.0  0.0  2.0  0.0  0.0  0.0
3  0.0  0.0  0.0  0.0  0.0  0.0
Process 1

s1x:1,s1y:1,s2x:3,s2y:3
y_i:2,y_f:4
iter: 0   erro: 0.020000000000000004

unew: 
      0    1    2    3    4    5
0  0.0  0.0  0.0  0.0  0.0  0.0
1  0.0  0.0  0.0  0.0  0.0  0.0
2  0.0  0.0  0.0  0.0  0.0  0.0
3  0.0  0.0  0.0  0.0  0.0  0.0
f: 
      0    1    2    3    4    5
0  0.0  0.0  0.0  0.0  0.0  0.0
1  0.0  0.0  0.0  0.0  0.0  0.0
2  0.0  0.0  0.0  0.0 -2.0  0.0
3  0.0  0.0  0.0  0.0  0.0  0.0
Process 0

s1x:1,s1y:1,s2x:3,s2y:3
y_i:0,y_f:2
iter: 1   erro: 0.008660254037844388

unew: 
      0    1     2    3    4    5
0  0.0  0.0  0.00  0.0  0.0  0.0
1  0.